# Multiple Regression

In questo notebook saranno utilizzati i dati delle vendite delle case in King County per predire i prezzi, utilizzando la multiple regression
Cosa Faremo :

* Utilizzo di funzioni in Graphlab per calcolare i coefficienti della regressione
* Dati i coefficienti, i predittori e i pesi, scrivere una funzione per calcolare la RSS.
* Guardare i coefficienti e comprendere il loro significato.
* Valutare i modelli tramite la RSS

# Import di  Graphlab create


In [1]:
import graphlab

# Caricamento dei dati di input
Consideriamo un dataset contentente i prezzi delle case vendute nella contea di King tra Maggio del 2014 e Maggio del 2015

In [34]:
# Scommentare la riga sotto per caricare il dataset
#
#sales = graphlab.SFrame.read_csv('data/kc_house_data.csv')

# Divido i dati in training set e test set

In [3]:
train_data,test_data = sales.random_split(.8,seed=0)

# Costruisci la multiple linear regression  

Consideriamo un modello di multiple linear regression che considera:
example_features = ['sqft_living', 'bedrooms', 'bathrooms'] sui dati di training:



In [4]:
example_features = ['sqft_living', 'bedrooms', 'bathrooms']
example_model = graphlab.linear_regression.create(train_data, target = 'price', 
                                                  features = example_features, 
                                                  validation_set = None)

<pre>Linear regression:</pre>

<pre>--------------------------------------------------------</pre>

<pre>Number of examples          : 17384</pre>

<pre>Number of features          : 3</pre>

<pre>Number of unpacked features : 3</pre>

...

<pre>+-----------+----------+--------------+--------------------+---------------+</pre>

<pre>| Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |</pre>

<pre>+-----------+----------+--------------+--------------------+---------------+</pre>

<pre>| 1         | 2        | 1.027018     | 4146407.590959     | 258679.804259 |</pre>

<pre>+-----------+----------+--------------+--------------------+---------------+</pre>

Estraiamo i coefficienti calcolati dal modello

In [5]:
example_weight_summary = example_model.get("coefficients")
print example_weight_summary

+-------------+-------+----------------+---------------+
|     name    | index |     value      |     stderr    |
+-------------+-------+----------------+---------------+
| (intercept) |  None | 87910.0692067  | 7873.33813675 |
| sqft_living |  None | 315.403441118  | 3.45570032294 |
|   bedrooms  |  None | -65080.2154416 | 2717.45685213 |
|  bathrooms  |  None | 6944.02087632  | 3923.11492813 |
+-------------+-------+----------------+---------------+
[4 rows x 4 columns]



In [6]:
example_model.show()

Canvas is accessible via web browser at the URL: http://localhost:49660/index.html
Opening Canvas in default web browser.


# Fare predizioni

Utilizziamo la funzione predict, già pronta in graphlab, per predire i risultati dei dati di training, basandosi sul modello creato

In [7]:
example_predictions = example_model.predict(train_data)
print example_predictions[0] # should be 271789.505878

271789.504278


In [8]:
train_data['price'].head()

dtype: float
Rows: 10
[221900.0, 538000.0, 180000.0, 604000.0, 510000.0, 1225000.0, 257500.0, 291850.0, 229500.0, 323000.0]

In [9]:
example_predictions.head()

dtype: float
Rows: 10
[271789.50427757535, 718880.3135271334, 207554.3088607718, 466612.0146607226, 436435.24571294733, 1568323.9522438196, 449210.37137116166, 237413.101781566, 461031.5689484328, 506141.9787859095]

In [10]:
e = (train_data['price'][0]-example_predictions[0])
print e

-49889.5042776


# Calcolo del RSS

Dato il modello, i dati e l'output, calcoliamo RSS

In [11]:
def get_residual_sum_of_squares(model, data, outcome):
    # Calcoliamo le prediction
    predictions = model.predict(data)
    # Calcolo degli errori
    residuals = predictions - outcome
    # calcolo della RSS
    RSS = (residuals * residuals).sum()
    return(RSS)    

Test della funzione sui dati di test

In [12]:
rss_example_train = get_residual_sum_of_squares(example_model, test_data, test_data['price'])
print rss_example_train # should be 2.7376153833e+14

2.73761534465e+14


# Creazione di nuove features

In [13]:
from math import log

E’ possibile considerare come nuove features anche trasformazioni delle feature esistenti con lo scopo di ottenere una migliore predizione.
Creiamo 4 nuove features :
* bedrooms_squared = (bedrooms*bedrooms). 
Questo permetterà di creare una distinzione netta tra le case con poche camere
da letto (es: 1) e quelle con molte camere (es: 4) in quanto 1^2 =1, ma 4^2 =16.

* bed_bath_rooms = bedrooms*bathrooms
E’ anche chiamata “feature interaction” ed è più alta quando ci sono sia tante
camere che tanti bagni

* log_sqft_living = log(sqft_living)
Fare il logaritmo ha l’effetto di portare i valori più grandi vicini e di espandere
quelli più piccoli

* lat_plus_long = lat + long

In [16]:
train_data['bedrooms_squared'] = train_data['bedrooms'].apply(lambda x: x**2)
test_data['bedrooms_squared'] = test_data['bedrooms'].apply(lambda x: x**2)

train_data['bed_plus_bath'] = train_data.apply(lambda x: x['bedrooms'] * x['bathrooms'])
test_data['bed_plus_bath'] = test_data.apply(lambda x: x['bedrooms'] * x['bathrooms'])

train_data['log_sqft_living'] = train_data['sqft_living'].apply(lambda x: log(x))
test_data['log_sqft_living'] = test_data['sqft_living'].apply(lambda x: log(x))

train_data['lat_plus_long'] = train_data.apply(lambda x: x['lat'] + x['long'])
test_data['lat_plus_long'] = test_data.apply(lambda x: x['lat'] + x['long'])

In [18]:
print test_data['bedrooms_squared'].mean()
print test_data['bed_plus_bath'].mean()
print test_data['log_sqft_living'].mean()
print test_data['lat_plus_long'].mean()

12.4466777016
7.50390163159
7.55027467965
-74.653333554


# Confronto tra modelli

Definisco 3 nuovi modelli, con le nuove features create.
* Modello 1: sqft_living, #bedrooms, #bathrooms, latitude and longitude
* Modello 2: Modello 1+ bed_bath_rooms
* Modello 3: Modello 2 + bedrooms_squared, log_sqft_living,lat_plus_long.

In [28]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_plus_bath']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

Ora che ho le features, posso imparare i parametri per i tre differenti modelli nel predire il prezzo utilizzando graphlab.linear_regression.create() e guardando il valore dei coefficienti. 

In [26]:
# Learn the three models: (don't forget to set validation_set = None)
model_1 = graphlab.linear_regression.create(train_data, target = 'price', features = model_1_features, validation_set = None)


<pre>Linear regression:</pre>

<pre>--------------------------------------------------------</pre>

<pre>Number of examples          : 17384</pre>

<pre>Number of features          : 6</pre>

<pre>Number of unpacked features : 6</pre>

...

<pre>--------------------------------------------------------</pre>

<pre>+-----------+----------+--------------+--------------------+---------------+</pre>

<pre>| Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |</pre>

<pre>+-----------+----------+--------------+--------------------+---------------+</pre>

<pre>| 1         | 2        | 0.040354     | 4014053.068747     | 235190.383243 |</pre>

In [29]:
model_2 = graphlab.linear_regression.create(train_data, target = 'price', features = model_2_features, validation_set = None)

<pre>Linear regression:</pre>

<pre>--------------------------------------------------------</pre>

<pre>Number of examples          : 17384</pre>

<pre>Number of features          : 6</pre>

<pre>Number of unpacked features : 6</pre>

...

<pre>--------------------------------------------------------</pre>

<pre>+-----------+----------+--------------+--------------------+---------------+</pre>

<pre>| Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |</pre>

<pre>+-----------+----------+--------------+--------------------+---------------+</pre>

<pre>| 1         | 2        | 0.040354     | 4014053.068747     | 235190.383243 |</pre>

In [30]:
model_3 = graphlab.linear_regression.create(train_data, target = 'price', features = model_3_features, validation_set = None)

<pre>Linear regression:</pre>

<pre>--------------------------------------------------------</pre>

<pre>Number of examples          : 17384</pre>

<pre>Number of features          : 9</pre>

<pre>Number of unpacked features : 9</pre>

<pre>Number of coefficients    : 10</pre>

<pre>Starting Newton Method</pre>

<pre>--------------------------------------------------------</pre>

<pre>+-----------+----------+--------------+--------------------+---------------+</pre>

In [31]:
#estrazione dei coefficienti per ciascun modello 
print 'model 1'
print model_1.get("coefficients")
print 'model 2'
print model_2.get("coefficients")
print 'model 3'
print model_3.get("coefficients")

model 1
+-------------+-------+----------------+---------------+
|     name    | index |     value      |     stderr    |
+-------------+-------+----------------+---------------+
| (intercept) |  None | -56141852.4692 | 1649928.35825 |
| sqft_living |  None | 310.263506724  | 3.18882062328 |
|   bedrooms  |  None | -59577.6052126 | 2487.27455846 |
|  bathrooms  |  None | 13812.4068763  | 3593.53440572 |
|     lat     |  None | 629863.119956  |  13120.68867  |
|     long    |  None | -214800.952332 | 13283.8203911 |
+-------------+-------+----------------+---------------+
[6 rows x 4 columns]

model 2
+---------------+-------+----------------+---------------+
|      name     | index |     value      |     stderr    |
+---------------+-------+----------------+---------------+
|  (intercept)  |  None | -54412036.4015 | 1650346.09777 |
|  sqft_living  |  None |  304.44950524  | 3.20216518282 |
|    bedrooms   |  None | -116366.742819 | 4805.53398548 |
|   bathrooms   |  None | -77972.10564

# Quale modello è il migliore? 

In [32]:
#Calcolo la RSS per ognuno dei tre modelli sui dati di TRAINING
print get_residual_sum_of_squares(model_1, train_data, train_data['price'])
print get_residual_sum_of_squares(model_2, train_data, train_data['price'])
print get_residual_sum_of_squares(model_3, train_data, train_data['price'])

9.7132387161e+14
9.61587552577e+14
9.05273774726e+14


In [33]:
#Calcolo la RSS per ognuno dei tre modelli sui dati di TEST
print get_residual_sum_of_squares(model_1, test_data, test_data['price'])
print get_residual_sum_of_squares(model_2, test_data, test_data['price'])
print get_residual_sum_of_squares(model_3, test_data, test_data['price'])

2.26566318597e+14
2.24366990581e+14
2.51824565049e+14


** Il modello migliore è model_2 poichè ha RSS sui test_data minore ** 